In [40]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
import warnings
warnings.filterwarnings('ignore')

LOADING DATA

In [41]:
df = pd.DataFrame(data.data)
df.columns = data.feature_names
df['result'] = data.target
df.shape

(569, 31)

DATA CHECK

In [42]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,result
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


CHECKING THE DEPENDENT DATA FOR SAMPLING RATIO

In [43]:
df['result'].value_counts()
# Data sampling is fine 

result
1    357
0    212
Name: count, dtype: int64

DEFINE DEPENDENT AND INDEPENDENT FEATURES

In [44]:
X = df.drop(['result'],axis=1)
y = df['result']

A Quick Analysis on model fit on data

In [45]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
# You can still pass hyper-parameter in here to avid Overfitting and Generalization : C = n-value, max_iter = n-value
from sklearn.model_selection import cross_val_score
cvs = cross_val_score(lr,X,y,scoring = 'accuracy',cv =10)
print(cvs)
# Mean values
print(np.mean(cvs)) # 94%, so basically if a data as no issues and pre-processing is done fine even when hyper-parameter is applied you get asame value

[0.92982456 0.92982456 0.92982456 0.92982456 0.94736842 0.96491228
 0.92982456 0.94736842 0.92982456 0.96428571]
0.9402882205513784


TESTING MODEL USING SPLIT

In [46]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test = tts(X,y,test_size=0.33,random_state=0)
print(f"X_train:{X_train.shape},X_test :{X_test.shape},y_train:{y_train.shape},y_test:{y_test.shape}")
lr.fit(X_train,y_train)
predicted = lr.predict(X_test)

X_train:(381, 30),X_test :(188, 30),y_train:(381,),y_test:(188,)


ERROR METRICS IN CLASSIFICAITON ACCURACY,PRECISION,RECALL,F1,CONFUSION, CLASSIFICATION REPORT

In [47]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report
accuracy = accuracy_score(predicted,y_test)
print(accuracy)
precision = precision_score(predicted,y_test)
print(precision)
recall = recall_score(predicted,y_test)
print(recall)
f1 = f1_score(predicted,y_test)
print(f1)
con = confusion_matrix(predicted,y_test)
print(con)
class_repo = classification_report(predicted,y_test)
print(class_repo)

0.9574468085106383
0.9504132231404959
0.9829059829059829
0.9663865546218487
[[ 65   6]
 [  2 115]]
              precision    recall  f1-score   support

           0       0.97      0.92      0.94        71
           1       0.95      0.98      0.97       117

    accuracy                           0.96       188
   macro avg       0.96      0.95      0.95       188
weighted avg       0.96      0.96      0.96       188



CLASSIFICAITON WITH HYPER-PARAMETER

In [48]:
# c and max_iter
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression(C=10,max_iter=40)
gs = GridSearchCV(lr,param_grid=[{'C' : [1,5,10,20]},{'max_iter' : [100,200,150]}],scoring='accuracy',cv=10)
gs.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(C=10, max_iter=40),
             param_grid=[{'C': [1, 5, 10, 20]}, {'max_iter': [100, 200, 150]}],
             scoring='accuracy')

In [49]:
print(gs.best_estimator_)
print(gs.best_params_)
print(gs.best_score_)

LogisticRegression(C=10, max_iter=200)
{'max_iter': 200}
0.9500674763832657


In [50]:
pred= gs.predict(X_test)
# measuing for accuracy and confusmatric
acc = accuracy_score(pred,y_test)
print(acc)

0.9521276595744681


In [51]:
con = confusion_matrix(pred,y_test)
print(con)

[[ 66   8]
 [  1 113]]


In [52]:
clas_repo = classification_report(pred,y_test)
print(class_repo)

              precision    recall  f1-score   support

           0       0.97      0.92      0.94        71
           1       0.95      0.98      0.97       117

    accuracy                           0.96       188
   macro avg       0.96      0.95      0.95       188
weighted avg       0.96      0.96      0.96       188

